# P5 (Використання та охорона земель, природних ресурсів і охорона довкілля): 0/4

In [1]:
import pandas as pd
from src.utils import normalize_parameter, save_data
from src.utils import PATH_RAW, PATH_INTERIM

## p5_01
### обсяги виявлених незаконних вирубок лісів
## p5_02
### частка земель лісового фонду, на які оформлені правовстановлюючі документи

In [2]:
agg_columns = [
    "Загальна площа земель лісового фонду, га", 
    "Площа земель лісового фонду, на які оформлені правовстановлюючі документи, га",
    "Обсяги виявлених незаконних рубок, м.куб."
]

df_P05_005 = pd.read_excel(PATH_INTERIM / "P5" / "P05_005.xlsx")
df_P05_005_fin = df_P05_005.groupby("region", as_index=False)[agg_columns].sum()
# df_P05_005_fin = df_P05_005_fin.rename(columns={'Обсяги виявлених незаконних рубок, м.куб.':'p5_01_raw'})
df_P05_005_fin["p5_01_raw"] = (
    df_P05_005_fin["Обсяги виявлених незаконних рубок, м.куб."] / df_P05_005_fin["Загальна площа земель лісового фонду, га"]
)
df_P05_005_fin["p5_02_raw"] = (
    df_P05_005_fin["Площа земель лісового фонду, на які оформлені правовстановлюючі документи, га"] / df_P05_005_fin["Загальна площа земель лісового фонду, га"]
)


df_P05_005_fin["p5_01"] = normalize_parameter(df_P05_005_fin["p5_01_raw"], min_bound=0, max_bound=1)
# Житомирська перевірити коли будуть усі дані
# можливо слід порівняти з минулим кварталом -- щось тут завелика площа з оформленими документами
df_P05_005_fin["p5_02"] = normalize_parameter(df_P05_005_fin["p5_02_raw"].clip(None, 1), min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.0006458690636220323, 0.009177197499319459); normalization_bounds=(0, 1)
feature_range=(0, 1); fill_na=True; array_bounds=(0.117512055058741, 1.0); normalization_bounds=(0, 1)


## p5_03
### площа об'єктів природно-заповідного фонду області відносно загальної площі області
## p5_07
### частка площа об'єктів природно-заповідного фонду місцевого значення області, на які оформлені правовстановлюючі документи

In [3]:
area = (
    pd
    .read_excel(PATH_RAW / "P99" / "area.xlsx")
    .set_index("Регіон ")["Площа (га)"]
    .to_dict()
)

df_P05_012 = pd.read_excel(PATH_INTERIM / "P5" / "P05_012.xlsx")
df_P05_012["Площа загальна (га)"] = df_P05_012["region"].map(area)

df_P05_012["p5_03_raw"] = (
    df_P05_012["Площа заповідної території (га)"] /
    df_P05_012["Площа загальна (га)"]
)
df_P05_012["p5_07_raw"] = df_P05_012["Частка затведжених проєктів землеустрою"]

df_P05_012["p5_03"] = normalize_parameter(df_P05_012["p5_03_raw"], min_bound=0)
df_P05_012["p5_07"] = normalize_parameter(df_P05_012["p5_07_raw"], min_bound=0, max_bound=1)
# normalize_parameter(df_P05_012, "Частка створених спеціальних адміністрацій?", "p5_03_адміністрації")

feature_range=(0, 1); fill_na=True; array_bounds=(0.0267414977876106, 0.05650781643138624); normalization_bounds=(0, 0.05650781643138624)
feature_range=(0, 1); fill_na=True; array_bounds=(0.009259259259259259, 0.2318339100346021); normalization_bounds=(0, 1)


## Група 1: p5_01-p5_02, незаконні вирубки
## Група 2: p5_03/p5_07, природно-заповідний фонд

In [4]:
dict_weights = {
    'p5_01':1,
    'p5_02':1,
    'p5_03':1,
    'p5_06':1,
    'p5_07':1,
}

sources = [df_P05_005_fin, df_P05_012]
save_data(sources, dict_weights, 'P5', show_results=True)

,region,p5_01,p5_02,p5_03,p5_07,P5
0,Вінницька,0.001240,0.957144,NaN,NaN,NaN
1,Волинська,0.001202,0.559946,0.899640,0.016438,2.954453
2,Дніпропетровська,NaN,NaN,NaN,NaN,NaN
3,Донецька,NaN,NaN,NaN,NaN,NaN
4,Житомирська,0.001879,1.000000,0.473235,0.009259,2.968748
5,Закарпатська,NaN,NaN,NaN,NaN,NaN
6,Запорізька,NaN,NaN,NaN,NaN,NaN
7,Івано-Франківська,0.001177,0.117512,0.985533,0.031373,2.271190
8,Київська,NaN,NaN,NaN,NaN,NaN
9,Кіровоградська,NaN,NaN,NaN,NaN,NaN
